In [1]:
# Project Overview: Create Relevant Spotify Playlist with Python
# Pre-work: Set up your Account & Register your application @ https://developer.spotify.com/documentation/web-api/quick-start/
# pip install requirements.txt
# 0. Declare name of playlist to be created
# 1. Import Packages and establish creds to connect to Spotify
# 2. Get raw Spotify data of Liked (Hearted) songs into a dataframe
# 3. Add audio features
# 4. Add genre data
# 5. Create genre score
### Repeat process (steps 2-5) for disliked songs (based on dislike playlist) ###
# 6. Create playlist df (to access disliked playlist)
# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
# ....
# 10. Create playlist

In [2]:
# 0. Declare name of playlist to be created
pred_like_playlist_name = "GenreScore>20_20200911"
genre_threshold = 21

In [3]:
#### ALL FUNCTIONS BELOW ####

# 1. Import Packages and establish creds to connect to Spotify
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import requests
import math
import time
import pandas as pd
from IPython.display import display
from pandas.io.json import json_normalize
import numpy as np 
#import matplotlib.pyplot as plt  
import inspect
import pickle
from datetime import datetime, timedelta

def define_scope():
    global username, scope, token, sp, user, headers
    username = os.environ['SPOTIPY_USERNAME']
    scope = '''
    playlist-modify-private 
    playlist-modify-public 
    playlist-read-collaborative 
    playlist-read-private
    user-follow-modify 
    user-follow-read 
    user-library-modify 
    user-library-read 
    user-modify-playback-state 
    user-read-currently-playing 
    user-read-email 
    user-read-playback-state 
    user-read-private 
    user-read-recently-played 
    user-top-read'''
    try:
        token = util.prompt_for_user_token(username, scope)
    except (AttributeError, JSONDecodeError):
        os.remove(f".cache-{username}")
        token = util.prompt_for_user_token(username, scope)
    sp = spotipy.Spotify(auth=token)
    user = sp.current_user()
    headers = {'Authorization': "Bearer {}".format(token)}
    return

# 2. Get raw Spotify data of Liked (i.e. Hearted) songs into a dataframe
# 2a. Get raw data of liked songs into a list
def req(url,headers):
    r = requests.get(url, headers=headers)
    return json.loads(r.text)

def get_raw_liked_song_list():
    print('Get_raw_liked_song_list')
    offset = 0
    url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}"

    liked_song_count = req(url,headers=headers)["total"]
    print("Liked song count: ", liked_song_count)

    raw_liked_songs = []
    for i in range(int(math.ceil(liked_song_count/50.0))):
        offset = 50*i
        raw_liked_songs.extend(req(f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}",headers=headers)["items"])

    print(f">> raw liked songs['0'] \n>> {raw_liked_songs[0]}")
    return raw_liked_songs

# 2b. transform liked song raw data list into df
# parse out data from Liked Songs' raw data
def song_metadata_to_df(raw_songs):
    print('Transforming song raw data list into df\n')
    define_scope()
    # create empty df
    df = pd.DataFrame(columns = [
        'album', 
        'artist_uri', 
        'artist',
        'track',
        'url',
        'popularity',
        'added_at',
        'release_date',
        'release_year',
        'uri'
        ])
    # parse out songs 100 at a time
    for i in range(0, len(raw_songs)):
        if (i/100).is_integer():
            print(f"{i}/{len(raw_songs)} parsed")
        try:
            album = raw_songs[i]['track']['album']['name']
            artist_uri = raw_songs[i]['track']['artists'][0]['uri'].split(':')[2]
            artist = raw_songs[i]['track']['album']['artists'][0]['name']
            track = raw_songs[i]['track']['name']
            url = raw_songs[i]['track']['external_urls']['spotify']
            popularity = raw_songs[i]['track']['popularity']
            added_at = raw_songs[i]['added_at']
            release_date = raw_songs[i]['track']['album']['release_date']
            release_year = release_date.split('-')[0]
            uri = raw_songs[i]['track']['uri']
            df = df.append(pd.DataFrame([[
                album, 
                artist_uri,
                artist, 
                track, 
                url, 
                popularity, 
                added_at, 
                release_date, 
                release_year,
                uri
                ]], columns = [
                'album', 
                'artist_uri', 
                'artist',
                'track',
                'url',
                'popularity',
                'added_at',
                'release_date',
                'release_year',
                'uri'
                ]))
        except:
            pass
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

#3. Add audio features 
# Combine liked song df with audio feature df
def add_audio_feats(df):
    print('Combining song_df with audio_feature_df')
    # get list of song uris
    song_uri_list = df.index.to_list()
    audio_feature_df = pd.DataFrame()
    for i in range(0,math.ceil(len(song_uri_list)/100)):
        define_scope()
        features = sp.audio_features(song_uri_list[(i)*100:100*(i+1)])
        audio_feature_df = audio_feature_df.append(pd.DataFrame(features))
    audio_feature_df.set_index('uri', inplace=True)
    # merge this 
    liked_song_df = df.merge(audio_feature_df, on = 'uri')
    display(liked_song_df.head())
    return liked_song_df.drop_duplicates()

#4. Add genre data
# 4a. Get artist_uris -> get genres
def get_genres(df):
    print('Getting genres\n')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(len(artist_uris), ' distinct artists')
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            print(f"{i}/{len(artist_uris)} parsed")
        try:
            g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
        except:
            define_scope()
            try:
                g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
            except:
                pass
    # genres_df's artist_uri column stores a list of genres per artist uri
    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    
    print('\ngenre_df')
    display(genres_df.head())
    return genres_df

# There is a list of genres for one artist that must be broken out
def explode_genres(df):
    genre_exploded_df = df.explode('genre list')
    
    print('\ngenre_exploded_df')
    display(genre_exploded_df.head())
    return genre_exploded_df

# Add these genres back to the df for each song
def add_genres(df, genre_exploded_df):
    # the following does not work oddly enough
    # liked_song_df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri', inplace = True)
    df = df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri')
    df.rename(columns={'genre list': 'genre'}, inplace=True)
    
    print('Merging genres into song_df')
    display(df.head())
    return df

#5. Create Genre Score

# Get genre counts for liked songs - the higher the count, the higher the score
def get_genre_counts(genre_exploded_df):
    print('Getting genre counts')
    # establish genre counts (which will determine score)
    genre_count_df = pd.DataFrame(genre_exploded_df['genre list'].value_counts()).reset_index()
    genre_count_df.columns = ['genre','genre count']
    print('\ngenre_count_df.head()')
    display(genre_count_df.head())
    return genre_count_df

# Merge the genre counts with the song df and sum the genre column
def establish_genre_score(genre_count_df, df):
    print('\nMerging genre counts with song_df')
    df = df.reset_index().merge(genre_count_df , on = 'genre',how = 'left')
    df.drop(columns=['genre'], inplace=True)
    display(df.head())

    groupby_cols = df.columns.to_list()[:-1]
    
    print('\nSumming genre column')
    df = df.groupby(groupby_cols).sum().sort_values('genre count', ascending=False)
    df = df.rename({'genre count': 'genre score'}, axis=1).reset_index() 
    # reset index after summing - otherwise everything prior to sum will become index
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

# 6. Create playlist df (to access disliked playlist)
# get playlist raw data (for dislikes)
def get_playlist_uris(playlists_raw):
    df = pd.DataFrame()
    for i in range(0, len(playlists_raw)):
        try:
            playlist = playlists_raw[i]['name']
        except:
            playlist = ''
        try:
            playlist_uri = playlists_raw[i]['uri']
        except:
            playlist_uri = ''
        try:
            song_count = playlists_raw[i]['tracks']['total']
        except:
            song_count = 0
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    print('playlist_df')
    display(df)
    return df

def get_raw_disliked_song_list():
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    display(playlists_df)
    uri = playlists_df.loc['Dislikes']['playlist_uri']
    song_count = playlists_df.loc['Dislikes']['song_count']
    # extract data from Disliked songs
    dislike_raw_list = []
    for i in range(int(math.ceil(song_count/100.0))):
        dislike_raw_list.extend(sp.user_playlist_tracks(uri.split(':')[1], uri.split(':')[2], offset=i*100)['items'])
    print('Getting raw disliked song list')
    return dislike_raw_list

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
def combine_dfs_to_create_my_songs_df(df1,df2, save_destination):
    print('Combining dfs to create my_songs_df and storing as pickle file')
    df1['Liked'] = 1
    df2['Liked'] = 0
    my_songs_df = pd.concat([df1,df2])
    my_songs_df.to_csv(f"{save_destination}/my_songs.csv")
    my_songs_df.to_pickle(f"{save_destination}/my_songs.pkl")
    display(my_songs_df.head())
    print('successfully saved to .csv and .pkl files in save_destination')

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
def get_featured_playlist_uris(country, timestamp):
    print('Getting raw_featured_playlist_df')
    url = f"https://api.spotify.com/v1/browse/featured-playlists?limit=50&offset=0&country={country}&timestamp={timestamp}"
    featured_playlist_count = req(url,headers=headers)['playlists']['total']
    df = pd.DataFrame()
    for i in range(0,featured_playlist_count):
        raw_featured_playlist = req(url,headers=headers)['playlists']['items']
        playlist = raw_featured_playlist[i]['name']
        playlist_uri = raw_featured_playlist[i]['uri']
        song_count = raw_featured_playlist[i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
def get_raw_featured_playlist_song_list(df):
    raw_featured_songs = []
    for uri in df.loc[:,'playlist_uri'].to_list():
        song_count = df.loc[df['playlist_uri']==uri,'song_count'][0]
        offset = 0
        url = f"https://api.spotify.com/v1/playlists/{uri.split(':')[2]}/tracks?limit=50&offset={offset}"
        for i in range(int(math.ceil(song_count/100.0))):
            offset = 100*i
            raw_featured_songs.extend(req(url,headers=headers)['items'])

    return raw_featured_songs

def create_playlist(pred_like_playlist_name, df, genre_score_threshold):
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    playlist_uri = get_playlist_uris(playlists_raw).loc[playlists_df.index == pred_like_playlist_name,'playlist_uri'][0].split(':')[2]
    uri_list = df.loc[df['genre score']>= genre_score_threshold].reset_index()['uri'].to_list()
    num_songs_to_add = len(uri_list)
    for i in range(int(math.ceil(num_songs_to_add/100.0))):
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={','.join(uri_list[i*100:(i+1)*100])}"
        requests.post(url, headers=headers)
    return

In [4]:
define_scope()
print('Scope defined\n****PART 1: Get Liked Songs Dataframe****\n')
raw_liked_song_list = get_raw_liked_song_list()
liked_song_df = song_metadata_to_df(raw_liked_song_list)
liked_song_df = add_audio_feats(liked_song_df)
genre_df = get_genres(liked_song_df)
genre_exploded_df = explode_genres(genre_df)
liked_song_df = add_genres(liked_song_df, genre_exploded_df)
liked_genre_count_df = get_genre_counts(genre_exploded_df)
liked_song_df = establish_genre_score(liked_genre_count_df, liked_song_df)

Scope defined
****PART 1: Get Liked Songs Dataframe****

Get_raw_liked_song_list
Liked song count:  622
>> raw liked songs['0'] 
>> {'added_at': '2020-09-07T19:06:26Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0wyMPXGfOuQzNR54ujR9Ix'}, 'href': 'https://api.spotify.com/v1/artists/0wyMPXGfOuQzNR54ujR9Ix', 'id': '0wyMPXGfOuQzNR54ujR9Ix', 'name': 'Caamp', 'type': 'artist', 'uri': 'spotify:artist:0wyMPXGfOuQzNR54ujR9Ix'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI',

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,67,2020-09-07T19:06:26Z,2016-03-08,2016
spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,53,2020-09-07T19:03:34Z,2000-09-05,2000
spotify:track:1Exrzr80BmEOw2rMcRnjwh,I Forget Myself (feat. Darlingside),2XdtmipGVPmA62ptDgX8QC,Henry Jamison,I Forget Myself (feat. Darlingside),https://open.spotify.com/track/1Exrzr80BmEOw2r...,63,2020-09-07T18:34:21Z,2020-04-03,2020
spotify:track:0szmjOw9XbtbQYWz0GvXSp,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Horse to Water,https://open.spotify.com/track/0szmjOw9XbtbQYW...,61,2020-09-07T18:31:38Z,2016-08-19,2016
spotify:track:1vG6jMgSoqT3zG9tuDrL2E,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Spirit Cold,https://open.spotify.com/track/1vG6jMgSoqT3zG9...,61,2020-09-07T18:23:09Z,2016-08-19,2016


Combining song_df with audio_feature_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,67,2020-09-07T19:06:26Z,2016-03-08,2016,0.571,...,0.014600,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4
spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,53,2020-09-07T19:03:34Z,2000-09-05,2000,0.487,...,0.000003,0.1080,0.306,132.200,audio_features,2Zcr8xMMZyNT5wMsj3SWmw,https://api.spotify.com/v1/tracks/2Zcr8xMMZyNT...,https://api.spotify.com/v1/audio-analysis/2Zcr...,296725,4
spotify:track:1Exrzr80BmEOw2rMcRnjwh,I Forget Myself (feat. Darlingside),2XdtmipGVPmA62ptDgX8QC,Henry Jamison,I Forget Myself (feat. Darlingside),https://open.spotify.com/track/1Exrzr80BmEOw2r...,63,2020-09-07T18:34:21Z,2020-04-03,2020,0.580,...,0.003750,0.0788,0.339,141.938,audio_features,1Exrzr80BmEOw2rMcRnjwh,https://api.spotify.com/v1/tracks/1Exrzr80BmEO...,https://api.spotify.com/v1/audio-analysis/1Exr...,212147,4
spotify:track:0szmjOw9XbtbQYWz0GvXSp,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Horse to Water,https://open.spotify.com/track/0szmjOw9XbtbQYW...,61,2020-09-07T18:31:38Z,2016-08-19,2016,0.517,...,0.069900,0.4980,0.510,96.964,audio_features,0szmjOw9XbtbQYWz0GvXSp,https://api.spotify.com/v1/tracks/0szmjOw9Xbtb...,https://api.spotify.com/v1/audio-analysis/0szm...,244960,4
spotify:track:1vG6jMgSoqT3zG9tuDrL2E,Neptune,1OVaGC0CDZaxjcPxclSNmp,Tall Heights,Spirit Cold,https://open.spotify.com/track/1vG6jMgSoqT3zG9...,61,2020-09-07T18:23:09Z,2016-08-19,2016,0.426,...,0.006520,0.1760,0.359,78.247,audio_features,1vG6jMgSoqT3zG9tuDrL2E,https://api.spotify.com/v1/tracks/1vG6jMgSoqT3...,https://api.spotify.com/v1/audio-analysis/1vG6...,222293,4


Getting genres

399  distinct artists
0/399 parsed
100/399 parsed
200/399 parsed
300/399 parsed

genre_df


...

,genre,genre count
0,indie pop,75
1,indie rock,73
2,stomp and holler,70
3,indie folk,64
4,new americana,51



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,67,2020-09-07T19:06:26Z,2016-03-08,2016,...,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4,64.0
1,spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,67,2020-09-07T19:06:26Z,2016-03-08,2016,...,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4,51.0
2,spotify:track:3HAgxyWGeJtIVabS2mTREt,Caamp,0wyMPXGfOuQzNR54ujR9Ix,Caamp,Vagabond,https://open.spotify.com/track/3HAgxyWGeJtIVab...,67,2020-09-07T19:06:26Z,2016-03-08,2016,...,0.0926,0.611,141.922,audio_features,3HAgxyWGeJtIVabS2mTREt,https://api.spotify.com/v1/tracks/3HAgxyWGeJtI...,https://api.spotify.com/v1/audio-analysis/3HAg...,129464,4,70.0
3,spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,53,2020-09-07T19:03:34Z,2000-09-05,2000,...,0.1080,0.306,132.200,audio_features,2Zcr8xMMZyNT5wMsj3SWmw,https://api.spotify.com/v1/tracks/2Zcr8xMMZyNT...,https://api.spotify.com/v1/audio-analysis/2Zcr...,296725,4,19.0
4,spotify:track:2Zcr8xMMZyNT5wMsj3SWmw,Heartbreaker,2qc41rNTtdLK0tV3mJn2Pm,Ryan Adams,Oh My Sweet Carolina,https://open.spotify.com/track/2Zcr8xMMZyNT5wM...,53,2020-09-07T19:03:34Z,2000-09-05,2000,...,0.1080,0.306,132.200,audio_features,2Zcr8xMMZyNT5wMsj3SWmw,https://api.spotify.com/v1/tracks/2Zcr8xMMZyNT...,https://api.spotify.com/v1/audio-analysis/2Zcr...,296725,4,17.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,53,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.0961,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,494.0
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,46,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.0979,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,494.0
spotify:track:7ywp9MWERblgkSMoMBugC0,Holiday,6RWjTQqILL7a1tQ0VapyLK,The Magnetic Fields,Strange Powers,https://open.spotify.com/track/7ywp9MWERblgkSM...,51,2019-11-03T17:42:52Z,1994,1994,0.668,...,0.1290,0.963,131.767,audio_features,7ywp9MWERblgkSMoMBugC0,https://api.spotify.com/v1/tracks/7ywp9MWERblg...,https://api.spotify.com/v1/audio-analysis/7ywp...,161613,4,457.0
spotify:track:0iTpQYzJnYgh7kIxyq8A2O,Veckatimest,2Jv5eshHtLycR6R8KQCdc4,Grizzly Bear,Two Weeks,https://open.spotify.com/track/0iTpQYzJnYgh7kI...,69,2019-11-03T17:35:40Z,2009-05-24,2009,0.537,...,0.0800,0.704,159.871,audio_features,0iTpQYzJnYgh7kIxyq8A2O,https://api.spotify.com/v1/tracks/0iTpQYzJnYgh...,https://api.spotify.com/v1/audio-analysis/0iTp...,243160,4,447.0
spotify:track:0szSOfZVM3G5mWI1ogQjr7,Painted Ruins,2Jv5eshHtLycR6R8KQCdc4,Grizzly Bear,Mourning Sound,https://open.spotify.com/track/0szSOfZVM3G5mWI...,53,2019-11-03T17:41:44Z,2017-08-18,2017,0.507,...,0.1220,0.441,128.015,audio_features,0szSOfZVM3G5mWI1ogQjr7,https://api.spotify.com/v1/tracks/0szSOfZVM3G5...,https://api.spotify.com/v1/audio-analysis/0szS...,262107,4,447.0


In [ ]:
print('\n****PART 2: Get Disliked Songs Dataframe****\n')
define_scope()
raw_disliked_song_list = get_raw_disliked_song_list()
disliked_song_df = song_metadata_to_df(raw_disliked_song_list)
disliked_song_df = add_audio_feats(disliked_song_df)
genre_df = get_genres(disliked_song_df)
genre_exploded_df = explode_genres(genre_df)
disliked_song_df = add_genres(disliked_song_df, genre_exploded_df)
disliked_genre_count_df = get_genre_counts(genre_exploded_df)
disliked_genre_count_df['genre count'] = disliked_genre_count_df['genre count'] * -1
disliked_song_df = establish_genre_score(disliked_genre_count_df, disliked_song_df)

In [ ]:
print('\n****PART 3: Combine liked and disliked songs into my_songs df and genre_count_df and save***\n')
my_songs_df = combine_dfs_to_create_my_songs_df(liked_song_df,disliked_song_df,'.')
genre_count_df = pd.concat([liked_genre_count_df, disliked_genre_count_df])
#my_songs_df = pd.read_pickle("./my_songs.pkl")

In [ ]:
print('\n****PART 4: Get songs from featured playlists and score them based on genre****\n')
playlist_df = get_featured_playlist_uris('US',(pd.Timestamp.now()-timedelta(days=0)).strftime('%Y-%m-%dT%H:%M:%S.%Z'))
raw_featured_playlist_songs_list = get_raw_featured_playlist_song_list(playlist_df)
featured_playlist_song_df = song_metadata_to_df(raw_featured_playlist_songs_list)
featured_playlist_song_df
#featured_playlist_song_df = add_audio_feats(featured_playlist_song_df)
genre_df = get_genres(featured_playlist_song_df)
genre_exploded_df = explode_genres(genre_df)
featured_playlist_song_df = add_genres(featured_playlist_song_df, genre_exploded_df)
featured_playlist_song_df = establish_genre_score(genre_count_df, featured_playlist_song_df)
print('Complete! Check Spotify to see if playlist was filled with songs.')

In [ ]:
create_playlist(pred_like_playlist_name, featured_playlist_song_df, genre_threshold)

In [ ]:
# define_scope()
# req('https://api.spotify.com/v1/me',headers=headers)